In [12]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Word2Vec, StringIndexer, VectorAssembler, DataFrame
from pyspark.sql.functions import col, array, explode, concat_ws, collect_list, udf, lit
import numpy as np 
from pyspark.sql.types import StringType, ArrayType, BooleanType
from datasketch import MinHash, MinHashLSH
from sklearn.cluster import KMeans
from numpy import average

import shutil
import os



# All functions used

In [13]:
def shingle(text, k):
    shingle_set = []
    for i in range(len(text)-k +1):
        shingle_set.append(text[i:i+k])
    return set(shingle_set)

def minhash_lsh(df, k_shingle, threshold):

    lsh = MinHashLSH(threshold=threshold, num_perm=128)
    minhashes = {}

    for features in df.collect():
        shingles = shingle(features["features"], k_shingle)
        m = MinHash(num_perm=128)
        for shingle_item in shingles:
            m.update(shingle_item.encode("utf8"))
        minhashes[int(features["user_id"])] = m
        lsh.insert(int(features["user_id"]), m)

    replacement_candidates = {}
    for key in lsh.keys: 
        replacement_candidates[key] = lsh.query(minhashes[key]) 

    #Key: New representative, value: Similar items
    return replacement_candidates

#Iteratively bucket unique processes together
def bucketing(replacement_candidates):
    visited_processes = set()
    new_process_dictionary = {}
    for key, values in replacement_candidates.items():
        new_values = []
        for value in values:
            if value not in visited_processes:
                visited_processes.add(value)
                new_values.append(value)
        if new_values:  # Only add non-empty lists
            new_process_dictionary[key] = sorted(new_values)
    return new_process_dictionary


def kmeans_clustering(df, n_clusters, max_iter):
    minhashes = []
    #for jaccard verification
    minhash_dict = {}
    user_ids = []
    final_buckets = {}
    for features in df.collect():
        shingles = shingle(features["features"], 5)
        m = MinHash(num_perm=128)
        for shingle_item in shingles:
            m.update(shingle_item.encode("utf8"))
        minhashes.append(m.hashvalues)
        minhash_dict[int(features["user_id"])] = m
        user_ids.append(int(features["user_id"]))

    kmeans = KMeans(n_clusters=n_clusters, max_iter=max_iter).fit(minhashes)

    user_clusters = dict(zip(user_ids, kmeans.labels_))
    final_buckets = {}
    for key, value in user_clusters.items():
        if value in final_buckets:
            final_buckets[value].append(key)
        else:
            final_buckets[value] = [key]

    return final_buckets, minhash_dict


#Get averege jaccard value per bucket

def get_averege_jaccard_sim(final_buckets, minhashes):
    sims = {}
    for key, value in final_buckets.items():
        for user_id_1 in final_buckets[key]:
            for user_id_2 in final_buckets[key]:
                if user_id_1 != user_id_2:
                    sig_1 = minhashes[int(user_id_1)]
                    sig_2 = minhashes[int(user_id_2)]
                    sim = MinHash.jaccard(sig_1, sig_2)
                    if key not in sims:
                        sims[key] = [sim]
                    else:
                        sims[key].append(sim)
    total_sum = 0
    total_count = 0
    sims = dict(sorted(sims.items()))
    for key, value in sims.items():
        avg_sim = average(value)
        print(key, avg_sim)
        total_sum += sum(value)
        total_count += len(value)
    
    overall_average = total_sum / total_count if total_count != 0 else 0
    print("Overall Average Jaccard Similarity:", overall_average)


def write_df(df,file_name):
    os.makedirs('Output', exist_ok=True)
    #temporary folder to save all the temporaty files created by write.csv
    os.makedirs('temp', exist_ok=True)
    df.write.csv('temp/temp_outoput', header=True, mode="overwrite")
    part_file = [f for f in os.listdir('temp/temp_outoput') if f.startswith("part-")][0]

    shutil.move(os.path.join('temp/temp_outoput', part_file), file_name)
    shutil.rmtree('temp/temp_outoput')
    shutil.rmtree('temp')

def output_part1(dataset,k,threshold):
    data = spark.read.csv(dataset, header=True, inferSchema=True)
    df = data.withColumn("arrayColumn", concat_ws("","from", "to")).withColumn("Minhash", lit(""))
    df_grouped = df.groupBy("user_id").agg(
                                            concat_ws("",collect_list("arrayColumn")).alias("features"))
    
    replacement_candidates = minhash_lsh(df_grouped,k,threshold)
    new_process_dictionary = bucketing(replacement_candidates)    

    user_ids = list(new_process_dictionary.keys())
    output_df = data.filter(df.user_id.isin(user_ids))
    final_df = output_df.coalesce(1)
    return write_df(final_df,'Output/part1Output.csv')

In [6]:

spark = SparkSession.builder.appName("spark_session_1").getOrCreate()

data = spark.read.csv("data/SDG_dataset2.csv", header=True, inferSchema=True)

df = data.withColumn("arrayColumn", concat_ws("","from", "to")).withColumn("Minhash", lit(""))

df_grouped = df.groupBy("user_id").agg(
    concat_ws("",collect_list("arrayColumn")).alias("features")
)

## Parameter-tuning for k-shingles

In [27]:
replacement_candidates = minhash_lsh(df_grouped,3, 0.95)
new_process_dictionary = bucketing(replacement_candidates)
print(f"Initial processes: {len(replacement_candidates)}")
print(f"After merging processes: {len(new_process_dictionary)}")

replacement_candidates = minhash_lsh(df_grouped,5,0.95)
new_process_dictionary = bucketing(replacement_candidates)
print(f"Initial processes: {len(replacement_candidates)}")
print(f"After merging processes: {len(new_process_dictionary)}")

replacement_candidates = minhash_lsh(df_grouped,7, 0.95)
new_process_dictionary = bucketing(replacement_candidates)
print(f"Initial processes: {len(replacement_candidates)}")
print(f"After merging processes: {len(new_process_dictionary)}")

Initial processes: 45641
After merging processes: 8295
Initial processes: 45641
After merging processes: 12488
Initial processes: 45641
After merging processes: 33645


## Parameter tuning for Kmeans

In [34]:
from sklearn.model_selection import train_test_split, GridSearchCV

minhashes = []
user_ids = []
final_buckets = {}
for features in df_grouped.collect():
    shingles = shingle(features["features"], 5)
    m = MinHash(num_perm=128)
    for shingle_item in shingles:
        m.update(shingle_item.encode("utf8"))
    minhashes.append(m.hashvalues)
    user_ids.append(int(features["user_id"]))

param_grid = {
    'n_clusters': [100, 250, 500, 1000],
    'max_iter': [100, 500, 1000],
}

kmeans = KMeans()

grid_search = GridSearchCV(kmeans, param_grid, cv=5)

grid_search.fit(minhashes)

best_param = grid_search.best_params_
best_model = grid_search.best_estimator_

print("The best parameters are: " , best_param )
print("The best model is: ", best_model)

The best parameters are:  {'max_iter': 100, 'n_clusters': 500}
The best model is:  KMeans(max_iter=100, n_clusters=500)


## STEP 1: Find and merge

In [5]:
replacement_candidates = minhash_lsh(df_grouped,7, 0.95)
new_process_dictionary = bucketing(replacement_candidates)
print(len(replacement_candidates))
print(len(new_process_dictionary))

45641
33645


In [11]:
output_part1("data/SDG_dataset2.csv",7,0.95)

Py4JJavaError: An error occurred while calling o33858.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.jav

## STEP 2: Find/cluster similar items

In [37]:
from sklearn.model_selection import train_test_split, GridSearchCV

users = []
for key in new_process_dictionary:
    users.append(key)

filtered_df = df_grouped[df_grouped['user_id'].isin(users)]

final_buckets, minhashes = kmeans_clustering(filtered_df,500,100)

## Verification

In [38]:
get_averege_jaccard_sim(final_buckets, minhashes)

0 0.7834201388888888
1 0.847677517961901
2 0.7447797619047619
3 0.7051776208026208
4 0.8617927376229084
5 0.7972745028409091
6 0.7836498001262361
7 0.9415954415954416
8 0.8053662455673759
9 0.8356819600499376
10 0.8339362026862027
11 0.7604993386243386
12 0.8120081018518519
13 0.723417467948718
14 1.0
15 0.8237791719948849
16 0.8545228687739463
17 0.8591193990042674
18 0.8710798012730943
19 0.7525192840189874
20 0.8004079693532818
21 0.7447916666666666
22 0.8426411290322581
23 0.8158482142857143
24 0.8285163849708171
25 0.8749687208277149
26 0.7049501050420168
27 0.8059927208835341
28 0.759171195652174
29 0.7497585338419406
30 0.8264365290034933
31 0.9097732843137255
32 0.8368202110389611
33 0.8504585597826086
34 0.801110197368421
35 0.8607062177280551
36 0.7909757653061225
37 0.8297011066763426
38 0.8109365816926242
39 0.7466061282467532
40 0.852827380952381
41 0.8078188259109311
42 0.7372295673076923
43 0.846875
44 0.8608469202898551
45 0.8153409090909091
46 0.8214492394179894
47 0.7

In [50]:
# indexer = StringIndexer(inputCol="features", outputCol="from_to_type_index")
# indexed_data = indexer.fit(actual_routes_feature).transform(actual_routes_feature)
# assembler = VectorAssembler(inputCols=["from_to_type_index"], outputCol="vector")
# actual_feature_data = assembler.transform(indexed_data)

# actual_feature_data.show()
# # def is_non_zero_vector(vector):
# #     return vector.numNonzeros() > 0

# # is_non_zero_vector_udf = udf(is_non_zero_vector, BooleanType())

# # filtered_data = actual_feature_data.filter(is_non_zero_vector_udf(col("vector")))


# mh = MinHashLSH(inputCol="vector", outputCol="hashes", numHashTables=5, seed=1003)
# model = mh.fit(actual_feature_data)

# #transformed_filtered_data = model.transform(actual_feature_data).head()
# test = model.approxNearestNeighbors()

# # transformed_filtered_data.show(truncate=False, n=50)


# #similar_items.show(truncate=False)

# def is_non_zero_vector(vector):
#     return vector.numNonzeros() > 0

# from collections import defaultdict

# representative_mapping = {}

# group_mapping = defaultdict(list)

# # Iterate over the user neighbors dictionary
# for user, neighbors in new_process_dictionary.items():
#     neighbors_sorted = tuple(sorted(neighbors))
#     if neighbors_sorted in representative_mapping:
#         representative = representative_mapping[neighbors_sorted]
#     else:
#         representative = neighbors_sorted[0]
#         for neighbor in neighbors_sorted:
#             representative_mapping[neighbor] = representative
    
#     representative_mapping[user] = representative
#     group_mapping[representative].append(user)

# new_user_neighbors = {}
# for representative, users in group_mapping.items():
#     new_user_neighbors[representative] = users

# #print(new_user_neighbors)
# new_users = []
# for key, value in new_user_neighbors.items():
#     new_users.append(key)

# print(len(new_users))
# filtered_df = df_grouped[df_grouped['user_id'].isin(new_users)]
# print(f"Amount of processes: {filtered_df.count()}")
#     shingles = shingle(features["features"], 5)
#     m = MinHash(num_perm=128)
#     for shingle_item in shingles:
#         m.update(shingle_item.encode("utf8"))
#     minhashes[int(features["user_id"])] = m
#     lsh.insert(int(features["user_id"]), m)
#     neigbours = lsh.query(m)
#     print(features["user_id"], neigbours)
#     final_buckets[features["user_id"]] = neigbours

# print(final_buckets)